In [3]:
import pandas as pd

In [5]:
train=pd.read_csv("C://Users//pedda//Downloads//playground-series-s5e4//train.csv")
test=pd.read_csv("C://Users//pedda//Downloads//playground-series-s5e4//test.csv")
sample_submission=pd.read_csv("C://Users//pedda//Downloads//playground-series-s5e4//sample_submission.csv")
                              

In [6]:
print(train.head())

   id     Podcast_Name Episode_Title  Episode_Length_minutes       Genre  \
0   0  Mystery Matters    Episode 98                     NaN  True Crime   
1   1    Joke Junction    Episode 26                  119.80      Comedy   
2   2   Study Sessions    Episode 16                   73.90   Education   
3   3   Digital Digest    Episode 45                   67.17  Technology   
4   4      Mind & Body    Episode 86                  110.51      Health   

   Host_Popularity_percentage Publication_Day Publication_Time  \
0                       74.81        Thursday            Night   
1                       66.95        Saturday        Afternoon   
2                       69.97         Tuesday          Evening   
3                       57.22          Monday          Morning   
4                       80.07          Monday        Afternoon   

   Guest_Popularity_percentage  Number_of_Ads Episode_Sentiment  \
0                          NaN            0.0          Positive   
1           

In [7]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 750000 entries, 0 to 749999
Data columns (total 12 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   id                           750000 non-null  int64  
 1   Podcast_Name                 750000 non-null  object 
 2   Episode_Title                750000 non-null  object 
 3   Episode_Length_minutes       662907 non-null  float64
 4   Genre                        750000 non-null  object 
 5   Host_Popularity_percentage   750000 non-null  float64
 6   Publication_Day              750000 non-null  object 
 7   Publication_Time             750000 non-null  object 
 8   Guest_Popularity_percentage  603970 non-null  float64
 9   Number_of_Ads                749999 non-null  float64
 10  Episode_Sentiment            750000 non-null  object 
 11  Listening_Time_minutes       750000 non-null  float64
dtypes: float64(5), int64(1), object(6)
memory usage: 68.7+ MB


In [8]:
train.describe()

,id,Episode_Length_minutes,Host_Popularity_percentage,Guest_Popularity_percentage,Number_of_Ads,Listening_Time_minutes
count,750000.000000,662907.000000,750000.000000,603970.000000,749999.000000,750000.000000
mean,374999.500000,64.504738,59.859901,52.236449,1.348855,45.437406
std,216506.495284,32.969603,22.873098,28.451241,1.151130,27.138306
min,0.000000,0.000000,1.300000,0.000000,0.000000,0.000000
25%,187499.750000,35.730000,39.410000,28.380000,0.000000,23.178350
50%,374999.500000,63.840000,60.050000,53.580000,1.000000,43.379460
75%,562499.250000,94.070000,79.530000,76.600000,2.000000,64.811580
max,749999.000000,325.240000,119.460000,119.910000,103.910000,119.970000


In [11]:
train.isnull().sum()

id                                  0
Podcast_Name                        0
Episode_Title                       0
Episode_Length_minutes          87093
Genre                               0
Host_Popularity_percentage          0
Publication_Day                     0
Publication_Time                    0
Guest_Popularity_percentage    146030
Number_of_Ads                       1
Episode_Sentiment                   0
Listening_Time_minutes              0
dtype: int64

In [15]:
test.isnull().sum()

id                                 0
Podcast_Name                       0
Episode_Title                      0
Episode_Length_minutes         28736
Genre                              0
Host_Popularity_percentage         0
Publication_Day                    0
Publication_Time                   0
Guest_Popularity_percentage    48832
Number_of_Ads                      0
Episode_Sentiment                  0
dtype: int64

In [17]:
# Drop Episode_Title (optional: high cardinality, maybe not informative)
train = train.drop(columns=['Episode_Title'])
test = test.drop(columns=['Episode_Title'])

In [19]:
# Save test IDs
test_ids = test['id']

In [21]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error,r2_score
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression

In [22]:
#combining train and test fro preprossing
train['is_train']=1
test['is_test']=0
combined=pd.concat([train,test],axis=0)

In [25]:
#Handling missing numeric values
num_cols=['Episode_Length_minutes','Host_Popularity_percentage','Guest_Popularity_percentage']
imputer=SimpleImputer(strategy='mean')
combined[num_cols]=imputer.fit_transform(combined[num_cols])

In [27]:
combined.isnull().sum()

id                                  0
Podcast_Name                        0
Episode_Length_minutes              0
Genre                               0
Host_Popularity_percentage          0
Publication_Day                     0
Publication_Time                    0
Guest_Popularity_percentage         0
Number_of_Ads                       1
Episode_Sentiment                   0
Listening_Time_minutes         250000
is_train                       250000
is_test                        750000
dtype: int64

In [29]:
#Encoding categorical features
cat_cols=['Podcast_Name','Genre','Publication_Day','Publication_Time','Episode_Sentiment']
for col in cat_cols :
    le=LabelEncoder()
    combined[col]=le.fit_transform(combined[col].astype(str))
    

In [30]:
#final features
features=['Podcast_Name','Episode_Length_minutes','Genre','Host_Popularity_percentage','Guest_Popularity_percentage','Publication_Day','Publication_Time','Number_of_Ads','Episode_Sentiment']


In [33]:
#split combined back
train_data=combined[combined['is_train']==1]
test_data=combined[combined['is_test']==0]


In [35]:
X_train=train_data[features]
y_train=train_data['Listening_Time_minutes']
X_test=test_data[features]

In [37]:
#model1=LinearRegression()
#print("Training started")
#model1.fit(X_train,y_train)
#print("Training completed")
combined.shape

(1000000, 13)

In [39]:
#train model
model=RandomForestRegressor(random_state=42,n_estimators=89,max_depth=20,n_jobs=1)
#model.fit(X_train,y_train)

In [41]:

X_sample = X_train.sample(1000, random_state=42)
y_sample = y_train.loc[X_sample.index]

model.fit(X_sample, y_sample)


RandomForestRegressor(max_depth=20, n_estimators=89, n_jobs=1, random_state=42)

In [43]:
X_medium = X_train.sample(10000, random_state=42)
y_medium = y_train.loc[X_medium.index]
model.fit(X_medium, y_medium)


RandomForestRegressor(max_depth=20, n_estimators=89, n_jobs=1, random_state=42)

In [44]:
model.fit(X_train,y_train)

RandomForestRegressor(max_depth=20, n_estimators=89, n_jobs=1, random_state=42)

In [46]:
#predection on test data
test_pred=model.predict(X_test)

In [47]:
#save to submission file
submission=pd.DataFrame({'id':test_ids,
                         'Listening_Time_minutes':test_pred
                        })


In [ ]:
#submission.to_csv("submission.csv",index=False)#n_estimatoes==55

In [55]:
#submission.to_csv("submission1.csv",index=False)#n_estimators=89&max_depth=15

In [48]:
submission.to_csv("submission2.csv",index=False)#n_estimators=89&max_depth=20

In [49]:
y_pred=model.predict(X_train)
rmse=mean_squared_error(y_train,y_pred,squared=False)
rmse

C:\Users\pedda\anaconda3\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


9.088018118333643